In [ ]:
## Write scripts to load data from Alphavantage to Pandas Dataframe
import numpy as np
import pandas as pd
import requests

company_data = {}

# companies is supposed to be a list of symbols of companies but im not too sure whether to physically create the list of 500 symbols or a way to access it from the API.
companies = ["IBM"]
for company in companies:

    company_data[company] = generate_daily(company)


In [2]:
def generate_daily(stocks):

    url = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=stocks&outputsize=compact&apikey=1QPXCY5WU9WFE7N6'
    r = requests.get(url)
    data = r.json()

    return data

In [ ]:
def generate_daily_augmented(stocks, categories):
    return None